# Z3 Solver Project Demo
Presentation purposes

*Svolto per la presentazione del progetto all'orale*

## Scenario 1: Toy Model
Simple model containing just a few sessions and courses, not intended to be realistic in any way

*Modello semplice che contiene qualche sessione e corsi, inteso solo per motivi di testing*

In [1]:
from model import TimetableScheduler

In [2]:
import itertools

In [3]:
toy_schedule = TimetableScheduler('sample_1', 8)

In [4]:
toy_schedule.start()

In [5]:
toy_schedule.add_constraints()

In [6]:
toy_schedule.solve()

TIME TABLE FAILED


In [7]:
c = toy_schedule.solver.unsat_core()

In [8]:
toy_schedule.end()

In [9]:
len(c)

611

In [10]:
probs = []
for l in c:
    print(str(l).split("%"))

['C2-3', '0', '301']
['C2-3', '0', '302']
['C2-3', '0', '304']
['C2-3', '0', '305']
['C2-3', '0', '306']
['C2-3', '0', '307']
['C2-3', '0', '308']
['C2-3', '0', '309']
['C2-3', '0', '310']
['C2-3', '0', '311']
['C2-3', '0', '312']
['C2-3', '0', '313']
['C2-3', '1', '301']
['C2-3', '1', '302']
['C2-3', '1', '304']
['C2-3', '1', '305']
['C2-3', '1', '306']
['C2-3', '1', '307']
['C2-3', '1', '308']
['C2-3', '1', '309']
['C2-3', '1', '310']
['C2-3', '1', '311']
['C2-3', '1', '312']
['C2-3', '1', '313']
['C2-3', '2', '301']
['C2-3', '2', '302']
['C2-3', '2', '304']
['C2-3', '2', '305']
['C2-3', '2', '306']
['C2-3', '2', '307']
['C2-3', '2', '308']
['C2-3', '2', '309']
['C2-3', '2', '310']
['C2-3', '2', '311']
['C2-3', '2', '312']
['C2-3', '2', '313']
['C2-3', '3', '301']
['C2-3', '3', '302']
['C2-3', '3', '304']
['C2-3', '3', '305']
['C2-3', '3', '306']
['C2-3', '3', '307']
['C2-3', '3', '308']
['C2-3', '3', '309']
['C2-3', '3', '310']
['C2-3', '3', '311']
['C2-3', '3', '312']
['C2-3', '3',

In [ ]:
toy_schedule.model

In [ ]:
timeslots_hashmap = {
    0:  "Monday 08:00-09:00",
    1:  "Monday 09:00-10:00",
    2:  "Monday 10:00-11:00",
    3:  "Monday 11:00-12:00",
    4:  "Monday 12:00-13:00",
    5:  "Monday 13:00-14:00",
    6:  "Monday 14:00-15:00",
    7:  "Monday 15:00-16:00",
    8:  "Tuesday 08:00-09:00",
    9:  "Tuesday 09:00-10:00",
    10: "Tuesday 10:00-11:00",
    11: "Tuesday 11:00-12:00",
    12: "Tuesday 12:00-13:00",
    13: "Tuesday 13:00-14:00",
    14: "Tuesday 14:00-15:00",
    15: "Tuesday 15:00-16:00",
    16: "Wednesday 08:00-09:00",
    17: "Wednesday 09:00-10:00",
    18: "Wednesday 10:00-11:00",
    19: "Wednesday 11:00-12:00",
    20: "Wednesday 12:00-13:00",
    21: "Wednesday 13:00-14:00",
    22: "Wednesday 14:00-15:00",
    23: "Wednesday 15:00-16:00",
    24: "Thursday 08:00-09:00",
    25: "Thursday 09:00-10:00",
    26: "Thursday 10:00-11:00",
    27: "Thursday 11:00-12:00",
    28: "Thursday 12:00-13:00",
    29: "Thursday 13:00-14:00",
    30: "Thursday 14:00-15:00",
    31: "Thursday 15:00-16:00",
    32: "Friday 08:00-09:00",
    33: "Friday 09:00-10:00",
    34: "Friday 10:00-11:00",
    35: "Friday 11:00-12:00",
    36: "Friday 12:00-13:00",
    37: "Friday 13:00-14:00",
    38: "Friday 14:00-15:00",
    39: "Friday 15:00-16:00",
    40: "Saturday 08:00-09:00",
    41: "Saturday 09:00-10:00",
    42: "Saturday 10:00-11:00",
    43: "Saturday 11:00-12:00",
    44: "Saturday 12:00-13:00",
    45: "Saturday 13:00-14:00",
    46: "Saturday 14:00-15:00",
    47: "Saturday 15:00-16:00",
}

In [ ]:
toy_schedule.indexes

{'Sessions': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'Courses': [101, 102, 103, 104, 105],
 'Professors': [1, 2, 3],
 'CdS': [201, 202, 203],
 'Rooms': [301, 302, 303, 304, 305, 306, 307, 308]}

In [ ]:
toy_schedule.database.execute_query("SELECT * FROM CdS").fetchall()

[(201, 'AIDA', 80),
 (202, 'MATEMATICA', 40),
 (203, 'INGEGNERIA INFORMATICA', 70)]

In [ ]:
courses = [i[0] for i in toy_schedule.database.execute_query("SELECT DISTINCT Courses.IDCourse FROM Courses, CourseCdS WHERE CourseCdS.IDCdS = 203 AND Courses.IDCourse = CourseCdS.IDCourse;").fetchall()]

In [ ]:
courses

[101, 102, 103]

In [ ]:
names = toy_schedule.database.get_names()

In [ ]:
names

{'Courses': {101: 'ANALISI 1',
  102: 'PROGRAMMAZIONE',
  103: 'SISTEMI OPERATIVI',
  104: 'GEOMETRIA',
  105: 'ANALISI 2'},
 'Professors': {1: 'Prof 1', 2: 'Prof 2', 3: 'Prof 3'},
 'CdS': {201: 'AIDA', 202: 'MATEMATICA', 203: 'INGEGNERIA INFORMATICA'},
 'Rooms': {301: 'Aula Morin',
  302: 'Aula 4B H2 Bis',
  303: 'Aula Ciamician',
  304: 'Laboratorio Informatico',
  305: 'Aula Magna H3',
  306: 'Edificio Economo Aula 1',
  307: 'Aula 3A H2 Bis',
  308: 'Aula I Edificio Tutankhamon'}}

In [ ]:
# stampa l'orario per AIDA

i = 0
for t in toy_schedule.T:
    i+=1

    for (S, A) in itertools.product(toy_schedule.indexes['Sessions'], toy_schedule.indexes['Rooms']):
        S_c = toy_schedule.database.get_course(S)
        if S_c in courses:
            if toy_schedule.model.evaluate(toy_schedule.X[S,t,A]):
                print(f"{timeslots_hashmap[t]}: Session {names['Courses'][toy_schedule.database.get_course(S)]}, Room {names['Rooms'][A]}")
                break

    else:
        print(f"{timeslots_hashmap[t]}: NONE")
        continue
    
    if (i % 8) == 0:
        print("============"*10)

AttributeError: 'NoneType' object has no attribute 'evaluate'

## Scenario 2: Next Semester's Courses for AIDA
Simplified model containing (estimated) sessions and courses for the next semester in AIDA.

*Modello semplificato che contiene i corsi del prossimo semestre del CdS triennale in IADA (2025-2026, Primo Semestre)*

## Scenario 3: High School Timetable
Model creating a time table for the lessons of a class in high school, the last 3 years

*Modello che crea un orario settimanale per le lezioni del trienno di un liceo linguistico, solo per una classe*